In [22]:
from collections import defaultdict,OrderedDict
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import glob

pd.set_option('display.max_columns', None)

import datetime
import numpy as np
from joblib import Parallel, delayed
from time import sleep
from utils import *
from sklearn.mixture import BayesianGaussianMixture
import scipy.stats as stats
import importlib
import utils
importlib.reload(utils)


<module 'utils' from '/Users/vc/Research/caratterizzazione-sistemi-innovativi/utils.py'>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams.update({'font.size': 14})

/var/folders/bh/5p84z3p520g9wd5sphzrzscc0000gn/T/ipykernel_10571/3674912074.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [203]:
def clean_text(s):
    s = str(s).lower()
    l_of_words = ['and ','&',',','//','\\','the ','of ',
                  'co-','vice','vice-','(cro)']
    for w in l_of_words:
        s=s.replace(w,'')
    if 'chief executive officer' in s:
        s=s.replace('chief executive officer','ceo')
    if 'founder ceo' in s:
        s=s.replace('founder ceo','ceo founder') 
    l = s.split(' ')
    try:
        l.remove('')
    except:
        dummy=1
    # if 'ceo' in l and any(s.endswith('founder') or s.startswith('founder') for s in l):
    #     return ' '.join(l)
    # else:
    #     return ''
    return ' '.join(l)

def group_roles(df, col_name, groups):
    """
    Group roles in a DataFrame column based on a dictionary of groups.
    
    Args:
        df (pandas.DataFrame): The input DataFrame.
        col_name (str): The name of the column containing the roles.
        groups (dict): A dictionary where keys are the group names and values are lists of roles to be included in that group.
        
    Returns:
        pandas.DataFrame: A new DataFrame with a new column 'group' containing the grouped roles.
    """
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    # Create an empty 'group' column
    df_copy['group'] = ''
    
    # Iterate over each group in the dictionary
    for group_name, roles in groups.items():
        # Find rows where the role matches any role in the group
        mask = df_copy[col_name].str.lower().isin(roles)
        
        # Set the 'group' column for matching rows
        df_copy.loc[mask, 'group'] += group_name + ', '
    
    # Remove the trailing comma and space from the 'group' column
    df_copy['group'] = df_copy['group'].str.rstrip(', ')
    
    return df_copy

def find_job_word(df,field,word):
    return df[df[field].str.contains(word, case=False)]

def abbreviate_title(title):
    words = title.split()
    t = False
    other_jobs = ['president','founder','director']

    if len(words) <= 3 :
        abbreviated = ''.join([word[0] for word in words])
        t=True
    
    if len(words) == 4 and any(word in words for word in other_jobs):
        job = list(set(words) & set(other_jobs))[0]
        words.remove(job)
        abbreviated = ''.join([word[0] for word in words])
        abbreviated += ' '+job
        t=True

    if t:
        return abbreviated
    else:
        return title
    
def abbreviate_title_round_2(title):
    words = title.split()
    t = False
    other_jobs = ['president','founder','director','executive']

    if len(words) > 2 :
        abbreviated = ''.join([word[0] for word in words])
        t=True
        
    
    if len(words) > 2 and any(word in words for word in other_jobs):
        job = list(set(words) & set(other_jobs))
        for j in job:
            words.remove(j)
        abbreviated = ''.join([word[0] for word in words])
        abbreviated += ' '.join(job)
        
        t=True

    if t:
        return abbreviated
    else:
        return title
    

In [3]:
df_org         = pd.read_csv('../data/organizations.csv')
df_jobs        = pd.read_csv('../data/jobs.csv')
df_people      = pd.read_csv('../data/people.csv')
# df_fund_rounds = pd.read_csv('../data/funding_rounds.csv')
# df_ipos        = pd.read_csv('../data/ipos.csv')

In [4]:
start = datetime.datetime(1970,1,1)
end   = datetime.datetime(2024,12,30)

In [176]:
pple_new_roles = df_jobs.copy()
pple_new_roles['cleaned_title'] = pple_new_roles['title'].apply(lambda x: clean_text(x))
sorted_counts = pple_new_roles.groupby('cleaned_title')['cleaned_title'].count().sort_values(ascending=False)
sorted_counts_df = sorted_counts.reset_index(name='count')

In [ ]:
pple_new_roles[pple_new_roles.cleaned_title=='chief technology officer founder']

In [179]:
chief_job_dataframe = (
    find_job_word(sorted_counts_df,'cleaned_title','chief')
    .groupby('cleaned_title')
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
    .rename(columns={'index': 'chief_jobs'})
)

In [204]:
cumulative_chief = (
    find_job_word(sorted_counts_df,'cleaned_title','chief')
    .reset_index()  # Reset the index to make it a column
    .assign(cumulative_count=lambda x: x['count'].cumsum())  # Add a new column with cumulative sum
)

norm = cumulative_chief['cumulative_count'].tail(1).values[0]
cumulative_chief['percentage_cumulative']=cumulative_chief['cumulative_count']/norm
cumulative_chief['abbreviated_cleaned_title']=cumulative_chief['cleaned_title'].apply(abbreviate_title)

cumulative_chief['abbreviated_cleaned_title_round2']=cumulative_chief['abbreviated_cleaned_title'].apply(abbreviate_title_round_2)

cumulative_chief[cumulative_chief['percentage_cumulative']<0.70]

,index,cleaned_title,count,cumulative_count,percentage_cumulative,abbreviated_cleaned_title,abbreviated_cleaned_title_round2
0,16,chief financial officer,22362,22362,0.109538,cfo,cfo
1,21,chief operating officer,17607,39969,0.195784,coo,coo
2,25,chief technology officer,13498,53467,0.261903,cto,cto
3,37,chief marketing officer,7674,61141,0.299494,cmo,cmo
4,55,chief revenue officer,4670,65811,0.322369,cro,cro
...,...,...,...,...,...,...,...
209,2202,chief staff to ceo,58,142666,0.698836,chief staff to ceo,cstc
210,2204,chief financial officer chief compliance officer,58,142724,0.699120,chief financial officer chief compliance officer,cfocco
211,2207,executive president chief technology officer,58,142782,0.699404,executive president chief technology officer,ctoexecutive president
212,2229,chief operating financial officer,58,142840,0.699688,chief operating financial officer,cofo


In [151]:
cumulative_chief[cumulative_chief.cleaned_title]

,index,cleaned_title,count,cumulative_count,percentage_cumulative
0,16,chief financial officer,22362,22362,0.109538
1,21,chief operating officer,17607,39969,0.195784
2,25,chief technology officer,13498,53467,0.261903
3,38,chief marketing officer,7674,61141,0.299494
4,58,chief product officer,4452,65593,0.321301
...,...,...,...,...,...
35199,499970,founder chief botanic alchemist,1,204144,0.999980
35200,499971,founder chief bot architect,1,204145,0.999985
35201,499972,founder chief boat washer,1,204146,0.999990
35202,499973,founder chief board directors,1,204147,0.999995


In [109]:
if sorted_counts_df['cleaned_title'].isin(['chief']).any():
    print("'investor' exists in the 'cleaned_title' column")
else:
    print("'investor' does not exist in the 'cleaned_title' column")

'investor' exists in the 'cleaned_title' column


In [150]:
three_letter_titles = (
    pple_new_roles[pple_new_roles['cleaned_title'].str.len() == 3]
    .groupby('cleaned_title')
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
    .rename(columns={'index': 'three_letter_title'})
)
cumulative_three_letter = (
    three_letter_titles
    .reset_index()  # Reset the index to make it a column
    .assign(cumulative_count=lambda x: x['count'].cumsum())  # Add a new column with cumulative sum
)

norm = cumulative_three_letter['cumulative_count'].tail(1).values[0]

cumulative_three_letter['percentage_cumulative']=cumulative_three_letter['cumulative_count']/norm

cumulative_three_letter.head(20)

,index,cleaned_title,count,cumulative_count,percentage_cumulative
0,104,ceo,163515,163515,0.622988
1,172,cto,32852,196367,0.748153
2,109,cfo,25114,221481,0.843837
3,147,coo,20459,241940,0.921785
4,133,cmo,6724,248664,0.947403
5,121,cio,2390,251054,0.956509
6,165,cso,1768,252822,0.963245
7,153,cpo,1613,254435,0.969391
8,93,cco,1482,255917,0.975037
9,160,cro,1272,257189,0.979883


In [106]:
groups = {
    'ExCo': ['ceo', 'cto', 'cfo', 'coo', 'cmo', 'cio', 
             'cso', 'cpo', 'cco', 'cro', 'svp', 'evp', 
             'cdo', 'cbo', 'cxo'],
    'Founder': ['founder'],
    'Engineering': ['software engineer', 'data scientist']
}

# Call the group_roles function
grouped_df = group_roles(df, 'cleaned_title', groups)

print(grouped_df)

262469